# Guardrails for HR Agent Systems

**Building Secure and Compliant AI Agents with LangChain 1.0**

**What you'll learn:**
- 🛡️ PII Detection and Protection
- ✅ Input Validation Guardrails
- 🔍 Output Validation Guardrails
- 👤 Human-in-the-Loop for Sensitive Operations
- 🚦 Rate Limiting and Throttling
- 📋 Custom Business Rule Enforcement
- 🔄 Combining Multiple Guardrails

**Real HR Use Cases:**
- Prevent salary information leakage
- Redact personal identifiable information (PII)
- Require approval for sensitive operations
- Block inappropriate requests
- Enforce compliance requirements

**Time:** 2-3 hours

## Setup: Install Dependencies

In [ ]:
# Install LangChain 1.0 and required packages
!pip install --pre -U langchain langchain-openai langgraph langchain-community
!pip install langgraph-checkpoint-sqlite

## Setup: Configure API Keys

In [ ]:
from google.colab import userdata
import os

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

print("✅ API Key configured!")

## Common Imports

In [ ]:
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from typing import Annotated
from langgraph.checkpoint.memory import InMemorySaver

print("✅ Imports loaded!")

---
# Part 1: Understanding Guardrails 🛡️

## What are Guardrails?

Guardrails are safety mechanisms that control agent behavior:

```
Without Guardrails:
User: "What's Priya's salary?"
Agent: "Priya earns ₹15,00,000 per year"  ❌ Data leak!

With Guardrails:
User: "What's Priya's salary?"
Agent: "I cannot share salary information."  ✅ Protected!
```

## Types of Guardrails

1. **Input Guardrails** - Validate before execution
2. **Output Guardrails** - Validate after execution
3. **PII Detection** - Protect sensitive data
4. **Human-in-the-Loop** - Require approval
5. **Rate Limiting** - Prevent abuse

---
# Part 2: HR Tools Setup

Let's create HR tools with sensitive data that needs protection.

In [ ]:
# HR Database (simulated)
HR_DATABASE = {
    "101": {
        "name": "Priya Sharma",
        "department": "Engineering",
        "position": "Senior Developer",
        "email": "priya.sharma@company.com",
        "phone": "+91-9876543210",
        "salary": "₹15,00,000",
        "aadhar": "1234-5678-9012",
        "leave_balance": 12
    },
    "102": {
        "name": "Rahul Verma",
        "department": "Engineering",
        "position": "Manager",
        "email": "rahul.verma@company.com",
        "phone": "+91-9876543211",
        "salary": "₹22,00,000",
        "aadhar": "2345-6789-0123",
        "leave_balance": 8
    },
    "103": {
        "name": "Anjali Patel",
        "department": "HR",
        "position": "Director",
        "email": "anjali.patel@company.com",
        "phone": "+91-9876543212",
        "salary": "₹28,00,000",
        "aadhar": "3456-7890-1234",
        "leave_balance": 15
    },
    "104": {
        "name": "Arjun Reddy",
        "department": "Sales",
        "position": "Team Lead",
        "email": "arjun.reddy@company.com",
        "phone": "+91-9876543213",
        "salary": "₹18,00,000",
        "aadhar": "4567-8901-2345",
        "leave_balance": 10
    },
    "105": {
        "name": "Sneha Gupta",
        "department": "Marketing",
        "position": "Specialist",
        "email": "sneha.gupta@company.com",
        "phone": "+91-9876543214",
        "salary": "₹12,00,000",
        "aadhar": "5678-9012-3456",
        "leave_balance": 5
    }
}

print("✅ HR Database loaded with 5 employees!")

In [ ]:
# Basic HR Tools (WITHOUT guardrails)

@tool
def get_employee_info(employee_id: Annotated[str, "Employee ID to look up"]) -> str:
    """Get basic employee information."""
    if employee_id not in HR_DATABASE:
        return f"Employee {employee_id} not found"
    
    emp = HR_DATABASE[employee_id]
    return f"""
Name: {emp['name']}
Department: {emp['department']}
Position: {emp['position']}
Email: {emp['email']}
Phone: {emp['phone']}
"""

@tool
def get_salary_info(employee_id: Annotated[str, "Employee ID"]) -> str:
    """Get employee salary information. ⚠️ SENSITIVE DATA!"""
    if employee_id not in HR_DATABASE:
        return f"Employee {employee_id} not found"
    
    emp = HR_DATABASE[employee_id]
    return f"{emp['name']}'s annual salary: {emp['salary']}"

@tool
def update_salary(
    employee_id: Annotated[str, "Employee ID"],
    new_salary: Annotated[str, "New salary amount"]
) -> str:
    """Update employee salary. ⚠️ SENSITIVE OPERATION!"""
    if employee_id not in HR_DATABASE:
        return f"Employee {employee_id} not found"
    
    old_salary = HR_DATABASE[employee_id]['salary']
    HR_DATABASE[employee_id]['salary'] = new_salary
    return f"Salary updated for {HR_DATABASE[employee_id]['name']}: {old_salary} → {new_salary}"

@tool
def check_leave_balance(employee_id: Annotated[str, "Employee ID"]) -> str:
    """Check leave balance for an employee."""
    if employee_id not in HR_DATABASE:
        return f"Employee {employee_id} not found"
    
    emp = HR_DATABASE[employee_id]
    return f"{emp['name']} has {emp['leave_balance']} days of leave remaining"

@tool
def delete_employee(employee_id: Annotated[str, "Employee ID to delete"]) -> str:
    """Delete an employee from the system. ⚠️ CRITICAL OPERATION!"""
    if employee_id not in HR_DATABASE:
        return f"Employee {employee_id} not found"
    
    emp_name = HR_DATABASE[employee_id]['name']
    del HR_DATABASE[employee_id]
    return f"❌ Employee {emp_name} (ID: {employee_id}) has been deleted from the system"

print("✅ HR Tools defined (without guardrails)")

### Test Agent Without Guardrails

In [ ]:
# Create agent WITHOUT guardrails
unsafe_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[get_employee_info, get_salary_info, check_leave_balance],
    system_prompt="You are an HR assistant. Answer all questions about employees."
)

print("=" * 70)
print("Testing UNSAFE Agent (No Guardrails)")
print("=" * 70 + "\n")

# Test 1: PII exposure
result = unsafe_agent.invoke({
    "messages": [{"role": "user", "content": "What's Priya's contact information?"}]
})
print("❌ PII Exposed:")
print(result['messages'][-1].content)

print("\n" + "=" * 70 + "\n")

# Test 2: Salary information leak
result = unsafe_agent.invoke({
    "messages": [{"role": "user", "content": "How much does employee 101 earn?"}]
})
print("❌ Salary Info Leaked:")
print(result['messages'][-1].content)

print("\n⚠️ Without guardrails, sensitive data is exposed!")

---
# Part 3: PII Detection Guardrails 🔒

**Goal:** Automatically detect and redact Personally Identifiable Information.

In [ ]:
import re
from langchain.agents import AgentState
from langchain_core.messages import BaseMessage, AIMessage, HumanMessage
from typing import Sequence

# PII Detection Patterns
PII_PATTERNS = {
    "email": r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b",
    "phone_india": r"\+91-\d{10}|\d{10}",
    "aadhar": r"\d{4}-\d{4}-\d{4}",
    "pan": r"[A-Z]{5}\d{4}[A-Z]",
    "salary": r"₹[\d,]+"
}

def detect_and_redact_pii(text: str, strategy: str = "redact") -> tuple[str, list[str]]:
    """
    Detect and redact PII from text.
    
    Args:
        text: Text to scan for PII
        strategy: "redact", "mask", or "hash"
    
    Returns:
        (redacted_text, list of detected PII types)
    """
    redacted_text = text
    detected_types = []
    
    for pii_type, pattern in PII_PATTERNS.items():
        matches = re.findall(pattern, text)
        if matches:
            detected_types.append(pii_type)
            for match in matches:
                if strategy == "redact":
                    replacement = f"[REDACTED_{pii_type.upper()}]"
                elif strategy == "mask":
                    replacement = match[:2] + "*" * (len(match) - 4) + match[-2:]
                else:  # hash
                    import hashlib
                    replacement = hashlib.md5(match.encode()).hexdigest()[:8]
                
                redacted_text = redacted_text.replace(match, replacement)
    
    return redacted_text, detected_types

# Test PII detection
test_text = """
Priya Sharma
Email: priya.sharma@company.com
Phone: +91-9876543210
Aadhar: 1234-5678-9012
Salary: ₹15,00,000
"""

print("Original Text:")
print(test_text)
print("\n" + "=" * 70 + "\n")

redacted, detected = detect_and_redact_pii(test_text, strategy="redact")
print("Redacted Text:")
print(redacted)
print(f"\nDetected PII Types: {', '.join(detected)}")
print("\n✅ PII detection working!")

## Implement PII Guardrail for Agent

In [ ]:
def pii_output_guardrail(state: AgentState) -> dict:
    """
    Post-model hook to redact PII from agent responses.
    Runs AFTER the agent generates a response.
    """
    messages = state.get("messages", [])
    if not messages:
        return {}
    
    # Get the last message (agent's response)
    last_message = messages[-1]
    
    if isinstance(last_message, AIMessage) and last_message.content:
        # Redact PII from the response
        redacted_content, detected_pii = detect_and_redact_pii(
            last_message.content,
            strategy="redact"
        )
        
        if detected_pii:
            print(f"⚠️ PII Detected and Redacted: {', '.join(detected_pii)}")
            # Create new message with redacted content
            new_message = AIMessage(
                content=redacted_content,
                id=last_message.id
            )
            # Replace the last message
            return {"messages": messages[:-1] + [new_message]}
    
    return {}

print("✅ PII guardrail function created!")

### Test Agent With PII Guardrail

In [ ]:
# Create agent WITH PII guardrail
pii_protected_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[get_employee_info, check_leave_balance],
    post_model_hook=pii_output_guardrail,  # Add PII protection!
    system_prompt="You are an HR assistant. Provide employee information when requested."
)

print("=" * 70)
print("Testing Agent WITH PII Guardrail")
print("=" * 70 + "\n")

result = pii_protected_agent.invoke({
    "messages": [{"role": "user", "content": "What's Priya's contact information?"}]
})

print("✅ PII Protected Response:")
print(result['messages'][-1].content)
print("\n✅ Sensitive data automatically redacted!")

---
# Part 4: Input Validation Guardrails ✅

**Goal:** Block inappropriate or malicious requests BEFORE agent execution.

In [ ]:
from langchain_core.exceptions import AgentException

# Define blocked keywords and patterns
BLOCKED_KEYWORDS = [
    "hack", "exploit", "bypass", "steal",
    "delete all", "drop table", "sql injection"
]

SENSITIVE_TOPICS = [
    "salary", "compensation", "pay", "wage",
    "fire", "terminate", "layoff"
]

def input_validation_guardrail(state: AgentState) -> dict:
    """
    Pre-model hook to validate user input.
    Runs BEFORE the agent processes the request.
    """
    messages = state.get("messages", [])
    if not messages:
        return {}
    
    # Get the last user message
    user_message = None
    for msg in reversed(messages):
        if isinstance(msg, HumanMessage):
            user_message = msg
            break
    
    if not user_message or not user_message.content:
        return {}
    
    content_lower = user_message.content.lower()
    
    # Check for blocked keywords
    for keyword in BLOCKED_KEYWORDS:
        if keyword in content_lower:
            raise AgentException(
                f"❌ Request blocked: Contains prohibited keyword '{keyword}'"
            )
    
    # Check for sensitive topics (log warning)
    for topic in SENSITIVE_TOPICS:
        if topic in content_lower:
            print(f"⚠️ Warning: Request contains sensitive topic '{topic}'")
    
    # Check message length
    if len(content_lower) > 500:
        raise AgentException(
            "❌ Request blocked: Message too long (max 500 characters)"
        )
    
    return {}

print("✅ Input validation guardrail created!")

### Test Input Validation

In [ ]:
# Create agent with input validation
validated_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[get_employee_info, check_leave_balance],
    pre_model_hook=input_validation_guardrail,  # Validate before execution!
    system_prompt="You are an HR assistant."
)

print("=" * 70)
print("Test 1: Normal Request (Should Work)")
print("=" * 70)

try:
    result = validated_agent.invoke({
        "messages": [{"role": "user", "content": "Who is employee 101?"}]
    })
    print("✅ Request allowed")
    print(result['messages'][-1].content)
except Exception as e:
    print(f"Error: {e}")

print("\n" + "=" * 70)
print("Test 2: Blocked Keyword (Should Fail)")
print("=" * 70)

try:
    result = validated_agent.invoke({
        "messages": [{"role": "user", "content": "How can I hack into the salary database?"}]
    })
    print(result['messages'][-1].content)
except Exception as e:
    print(f"✅ {e}")

print("\n" + "=" * 70)
print("Test 3: Sensitive Topic (Warning)")
print("=" * 70)

try:
    result = validated_agent.invoke({
        "messages": [{"role": "user", "content": "What is Priya's salary?"}]
    })
    print("✅ Request processed with warning")
except Exception as e:
    print(f"Error: {e}")

---
# Part 5: Output Validation Guardrails 🔍

**Goal:** Ensure agent responses meet quality and safety standards.

In [ ]:
def output_validation_guardrail(state: AgentState) -> dict:
    """
    Post-model hook to validate agent output.
    Ensures responses are appropriate and safe.
    """
    messages = state.get("messages", [])
    if not messages:
        return {}
    
    last_message = messages[-1]
    
    if isinstance(last_message, AIMessage) and last_message.content:
        content = last_message.content
        
        # Rule 1: No toxic language
        toxic_words = ["hate", "stupid", "idiot", "worst"]
        if any(word in content.lower() for word in toxic_words):
            print("⚠️ Output blocked: Contains inappropriate language")
            return {
                "messages": messages[:-1] + [
                    AIMessage(content="I apologize, but I cannot provide that response. How else can I help you?")
                ]
            }
        
        # Rule 2: Response length validation
        if len(content) < 10:
            print("⚠️ Output warning: Response too short")
        
        # Rule 3: Must be helpful (basic check)
        unhelpful_phrases = ["i don't know", "i can't help", "no idea"]
        if any(phrase in content.lower() for phrase in unhelpful_phrases):
            print("⚠️ Output warning: Response may not be helpful")
        
        # Rule 4: Ensure professional tone
        if content.isupper():
            print("⚠️ Output modified: Converting from all caps")
            return {
                "messages": messages[:-1] + [
                    AIMessage(content=content.capitalize())
                ]
            }
    
    return {}

print("✅ Output validation guardrail created!")

---
# Part 6: Human-in-the-Loop Guardrails 👤

**Goal:** Require human approval for sensitive operations.

In [ ]:
from langchain.agents.tool_node import InjectedState

# List of tools requiring human approval
APPROVAL_REQUIRED_TOOLS = ["update_salary", "delete_employee"]

def human_approval_required(tool_name: str) -> bool:
    """Check if a tool requires human approval."""
    return tool_name in APPROVAL_REQUIRED_TOOLS

def request_human_approval(action: str, details: str) -> bool:
    """
    Simulate human approval request.
    In production, this would integrate with a real approval system.
    """
    print("\n" + "=" * 70)
    print("🔔 HUMAN APPROVAL REQUIRED")
    print("=" * 70)
    print(f"Action: {action}")
    print(f"Details: {details}")
    print("=" * 70)
    
    # Simulate approval (in production, wait for actual human input)
    response = input("Approve this action? (yes/no): ").strip().lower()
    
    if response == "yes":
        print("✅ Action approved by human")
        return True
    else:
        print("❌ Action rejected by human")
        return False

# Wrap sensitive tools with approval
@tool
def update_salary_with_approval(
    employee_id: Annotated[str, "Employee ID"],
    new_salary: Annotated[str, "New salary amount"]
) -> str:
    """Update employee salary with human approval."""
    
    if employee_id not in HR_DATABASE:
        return f"Employee {employee_id} not found"
    
    emp = HR_DATABASE[employee_id]
    details = f"Update {emp['name']}'s salary from {emp['salary']} to {new_salary}"
    
    # Request approval
    approved = request_human_approval("Salary Update", details)
    
    if not approved:
        return "❌ Salary update cancelled - approval denied"
    
    # Proceed with update
    old_salary = emp['salary']
    HR_DATABASE[employee_id]['salary'] = new_salary
    return f"✅ Salary updated for {emp['name']}: {old_salary} → {new_salary}"

@tool
def delete_employee_with_approval(
    employee_id: Annotated[str, "Employee ID to delete"]
) -> str:
    """Delete employee with human approval."""
    
    if employee_id not in HR_DATABASE:
        return f"Employee {employee_id} not found"
    
    emp = HR_DATABASE[employee_id]
    details = f"Delete {emp['name']} (ID: {employee_id}) from {emp['department']} department"
    
    # Request approval
    approved = request_human_approval("Employee Deletion", details)
    
    if not approved:
        return "❌ Employee deletion cancelled - approval denied"
    
    # Proceed with deletion
    emp_name = emp['name']
    del HR_DATABASE[employee_id]
    return f"✅ Employee {emp_name} (ID: {employee_id}) deleted from system"

print("✅ Human-in-the-loop tools created!")
print("Note: In production, integrate with approval workflow system")

---
# Part 7: Rate Limiting Guardrails 🚦

**Goal:** Prevent abuse by limiting request frequency.

In [ ]:
from datetime import datetime, timedelta
from collections import defaultdict

class RateLimiter:
    """
    Simple rate limiter to prevent abuse.
    Tracks requests per user/session.
    """
    def __init__(self, max_requests: int = 10, window_seconds: int = 60):
        self.max_requests = max_requests
        self.window_seconds = window_seconds
        self.request_log = defaultdict(list)
    
    def is_allowed(self, user_id: str) -> tuple[bool, int]:
        """
        Check if request is allowed.
        Returns (allowed, remaining_requests)
        """
        now = datetime.now()
        cutoff = now - timedelta(seconds=self.window_seconds)
        
        # Remove old requests
        self.request_log[user_id] = [
            req_time for req_time in self.request_log[user_id]
            if req_time > cutoff
        ]
        
        current_count = len(self.request_log[user_id])
        
        if current_count >= self.max_requests:
            return False, 0
        
        # Log this request
        self.request_log[user_id].append(now)
        remaining = self.max_requests - (current_count + 1)
        
        return True, remaining

# Create rate limiter (10 requests per minute)
rate_limiter = RateLimiter(max_requests=10, window_seconds=60)

def rate_limiting_guardrail(state: AgentState) -> dict:
    """
    Pre-model hook for rate limiting.
    """
    # Get user ID from state (in production, use actual user session)
    user_id = state.get("user_id", "default_user")
    
    allowed, remaining = rate_limiter.is_allowed(user_id)
    
    if not allowed:
        raise AgentException(
            "❌ Rate limit exceeded. Please wait before making more requests."
        )
    
    if remaining <= 2:
        print(f"⚠️ Rate limit warning: Only {remaining} requests remaining in this window")
    
    return {}

print("✅ Rate limiting guardrail created!")

### Test Rate Limiting

In [ ]:
# Test rate limiter
print("Testing Rate Limiter:")
print("=" * 70)

user_id = "test_user_rajesh"

for i in range(12):
    allowed, remaining = rate_limiter.is_allowed(user_id)
    if allowed:
        print(f"✅ Request {i+1}: Allowed (Remaining: {remaining})")
    else:
        print(f"❌ Request {i+1}: Blocked (Rate limit exceeded!)")

print("\n✅ Rate limiter working correctly!")

---
# Part 8: Custom Business Rules Guardrails 📋

**Goal:** Enforce company-specific policies and compliance.

In [ ]:
# Business rules configuration
BUSINESS_RULES = {
    "max_salary_update_percent": 20,  # Max 20% salary increase
    "min_employment_days_for_deletion": 90,  # Can't delete employees < 90 days
    "allowed_departments": ["Engineering", "HR", "Sales", "Marketing"],
    "working_hours": {"start": 9, "end": 18},  # 9 AM to 6 PM
}

def enforce_business_rules(state: AgentState) -> dict:
    """
    Enforce company business rules.
    """
    # Rule 1: Check working hours for sensitive operations
    current_hour = datetime.now().hour
    working_hours = BUSINESS_RULES["working_hours"]
    
    if not (working_hours["start"] <= current_hour < working_hours["end"]):
        print(f"⚠️ Warning: Operation outside working hours ({working_hours['start']}:00-{working_hours['end']}:00)")
    
    # Rule 2: Validate tool calls against business rules
    messages = state.get("messages", [])
    
    # Additional rule checks can be added here
    
    return {}

# Create validation function for salary updates
def validate_salary_update(employee_id: str, new_salary_str: str) -> tuple[bool, str]:
    """
    Validate salary update against business rules.
    """
    if employee_id not in HR_DATABASE:
        return False, "Employee not found"
    
    emp = HR_DATABASE[employee_id]
    
    # Extract numeric values
    old_salary = int(emp['salary'].replace('₹', '').replace(',', ''))
    new_salary = int(new_salary_str.replace('₹', '').replace(',', ''))
    
    # Calculate percentage change
    percent_change = ((new_salary - old_salary) / old_salary) * 100
    
    max_increase = BUSINESS_RULES["max_salary_update_percent"]
    
    if percent_change > max_increase:
        return False, f"Salary increase of {percent_change:.1f}% exceeds maximum allowed {max_increase}%"
    
    if percent_change < -max_increase:
        return False, f"Salary decrease of {abs(percent_change):.1f}% exceeds maximum allowed {max_increase}%"
    
    return True, f"Salary change of {percent_change:.1f}% within acceptable range"

# Test business rules
print("Testing Business Rules Validation:")
print("=" * 70)

# Test 1: Valid salary update
valid, msg = validate_salary_update("101", "₹17,00,000")  # ~13% increase
print(f"Test 1 - Valid Update: {valid} - {msg}")

# Test 2: Invalid salary update (too high)
valid, msg = validate_salary_update("101", "₹25,00,000")  # ~67% increase
print(f"Test 2 - Invalid Update: {valid} - {msg}")

print("\n✅ Business rules validation working!")

---
# Part 9: Production Agent with All Guardrails 🛡️

**Combining all guardrails for maximum protection!**

In [ ]:
from typing import TypedDict

# Enhanced state with user context
class SecureHRState(AgentState):
    user_id: str = "default_user"
    user_role: str = "employee"  # "employee", "manager", "hr_admin"
    employee_id: str = ""

# Combined guardrail hook
def combined_pre_guardrails(state: SecureHRState) -> dict:
    """Run all pre-execution guardrails."""
    # 1. Rate limiting
    rate_limiting_guardrail(state)
    
    # 2. Input validation
    input_validation_guardrail(state)
    
    # 3. Business rules
    enforce_business_rules(state)
    
    return {}

def combined_post_guardrails(state: SecureHRState) -> dict:
    """Run all post-execution guardrails."""
    # 1. Output validation
    result1 = output_validation_guardrail(state)
    if result1:
        return result1
    
    # 2. PII redaction
    result2 = pii_output_guardrail(state)
    if result2:
        return result2
    
    return {}

# Create production agent with all guardrails
production_secure_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[
        get_employee_info,
        check_leave_balance,
        update_salary_with_approval,  # Human-in-the-loop version
    ],
    state_schema=SecureHRState,
    pre_model_hook=combined_pre_guardrails,
    post_model_hook=combined_post_guardrails,
    checkpointer=InMemorySaver(),
    system_prompt="""You are a secure HR assistant with strict compliance guardrails.
    
    Your capabilities:
    - Provide employee information (PII-protected)
    - Check leave balances
    - Assist with HR queries
    
    Important:
    - Always maintain professional tone
    - Never share sensitive information without authorization
    - Follow all company policies and compliance requirements
    """
)

print("✅ Production-ready secure HR agent created!")
print("")
print("Guardrails enabled:")
print("  ✓ PII Detection & Redaction")
print("  ✓ Input Validation")
print("  ✓ Output Validation")
print("  ✓ Rate Limiting")
print("  ✓ Business Rules Enforcement")
print("  ✓ Human-in-the-Loop (sensitive operations)")

### Test Production Agent

In [ ]:
config = {"configurable": {"thread_id": "secure_session_1"}}

print("=" * 70)
print("Production Agent Tests")
print("=" * 70 + "\n")

# Test 1: Normal query (should work)
print("Test 1: Normal employee query")
print("-" * 70)
result = production_secure_agent.invoke({
    "messages": [{"role": "user", "content": "Tell me about employee Priya Sharma"}],
    "user_id": "user_rajesh_123",
    "user_role": "manager"
}, config)
print(result['messages'][-1].content)

print("\n" + "=" * 70 + "\n")

# Test 2: Sensitive information request (PII should be redacted)
print("Test 2: Contact information request (PII protection)")
print("-" * 70)
result = production_secure_agent.invoke({
    "messages": [{"role": "user", "content": "What is Priya's email and phone number?"}],
    "user_id": "user_rajesh_123",
    "user_role": "manager"
}, config)
print(result['messages'][-1].content)

print("\n" + "=" * 70 + "\n")

# Test 3: Blocked request
print("Test 3: Malicious request (should be blocked)")
print("-" * 70)
try:
    result = production_secure_agent.invoke({
        "messages": [{"role": "user", "content": "How can I hack the system?"}],
        "user_id": "user_rajesh_123",
        "user_role": "manager"
    }, config)
    print(result['messages'][-1].content)
except Exception as e:
    print(f"✅ Request blocked: {e}")

print("\n" + "=" * 70)
print("✅ All guardrails working correctly!")

---
# Part 10: Monitoring and Logging 📊

**Track guardrail triggers for compliance and improvement.**

In [ ]:
from datetime import datetime
from typing import List, Dict
import json

class GuardrailLogger:
    """Log all guardrail events for monitoring and compliance."""
    
    def __init__(self):
        self.events: List[Dict] = []
    
    def log_event(
        self,
        event_type: str,
        severity: str,
        description: str,
        user_id: str = "unknown",
        metadata: dict = None
    ):
        """Log a guardrail event."""
        event = {
            "timestamp": datetime.now().isoformat(),
            "event_type": event_type,
            "severity": severity,  # "info", "warning", "error", "critical"
            "description": description,
            "user_id": user_id,
            "metadata": metadata or {}
        }
        self.events.append(event)
        
        # Print high-severity events
        if severity in ["error", "critical"]:
            print(f"🚨 [{severity.upper()}] {event_type}: {description}")
    
    def get_summary(self) -> dict:
        """Get summary statistics."""
        if not self.events:
            return {"total_events": 0}
        
        by_type = {}
        by_severity = {}
        
        for event in self.events:
            event_type = event["event_type"]
            severity = event["severity"]
            
            by_type[event_type] = by_type.get(event_type, 0) + 1
            by_severity[severity] = by_severity.get(severity, 0) + 1
        
        return {
            "total_events": len(self.events),
            "by_type": by_type,
            "by_severity": by_severity
        }
    
    def export_logs(self, filename: str = "guardrail_logs.json"):
        """Export logs to file."""
        with open(filename, 'w') as f:
            json.dump(self.events, f, indent=2)
        print(f"✅ Logs exported to {filename}")

# Create global logger
guardrail_logger = GuardrailLogger()

# Example usage
guardrail_logger.log_event(
    event_type="pii_detected",
    severity="warning",
    description="Email address redacted from response",
    user_id="user_kavya_456",
    metadata={"pii_types": ["email"]}
)

guardrail_logger.log_event(
    event_type="input_blocked",
    severity="error",
    description="Request blocked due to prohibited keyword",
    user_id="user_amit_789",
    metadata={"keyword": "hack"}
)

guardrail_logger.log_event(
    event_type="human_approval_requested",
    severity="info",
    description="Salary update requires approval",
    user_id="user_meera_321",
    metadata={"action": "update_salary"}
)

# Display summary
print("\n" + "=" * 70)
print("Guardrail Event Summary")
print("=" * 70)
summary = guardrail_logger.get_summary()
print(json.dumps(summary, indent=2))
print("\n✅ Logging system ready!")

---
# Summary & Best Practices

## Guardrails Comparison

| Guardrail Type | When to Use | Execution Point | Example Use Case |
|----------------|-------------|-----------------|------------------|
| **PII Detection** | Protect sensitive data | Post-execution | Redact emails, phones, Aadhar |
| **Input Validation** | Block bad requests | Pre-execution | Filter malicious queries |
| **Output Validation** | Ensure quality | Post-execution | Remove toxic language |
| **Human-in-the-Loop** | Sensitive operations | During tool execution | Approve salary changes |
| **Rate Limiting** | Prevent abuse | Pre-execution | Limit requests per user |
| **Business Rules** | Enforce policies | Pre/Post-execution | Validate salary increases |

## Implementation Checklist

✅ **Layer multiple guardrails** for defense in depth  
✅ **Log all guardrail events** for compliance and monitoring  
✅ **Test guardrails thoroughly** before production  
✅ **Review logs regularly** to improve guardrails  
✅ **Make guardrails configurable** per environment  
✅ **Document all guardrail rules** for team  
✅ **Set up alerts** for critical violations  

## Production Architecture

```
User Request
    ↓
[Rate Limiting] ────→ Log
    ↓
[Input Validation] ──→ Log
    ↓
[Business Rules] ────→ Log
    ↓
Agent Execution
    ↓
[Human Approval?] ───→ Notification
    ↓
[Output Validation] ─→ Log
    ↓
[PII Redaction] ─────→ Log
    ↓
Response to User
```

## Key Takeaways

1. **Defense in Depth**: Multiple guardrails provide better protection than any single guardrail
2. **Fail Secure**: When in doubt, block the request
3. **Log Everything**: Compliance requires detailed audit trails
4. **Test Thoroughly**: Guardrails must work under all conditions
5. **Monitor Continuously**: Review logs and adjust guardrails

## Common Pitfalls to Avoid

❌ **Don't**: Rely on a single guardrail  
✅ **Do**: Layer multiple complementary guardrails

❌ **Don't**: Ignore logging  
✅ **Do**: Log all events for compliance

❌ **Don't**: Hard-code guardrail rules  
✅ **Do**: Make rules configurable

❌ **Don't**: Skip testing edge cases  
✅ **Do**: Test adversarial inputs

## Next Steps

- Integrate with production logging systems
- Add model-based guardrails (LLM safety checks)
- Implement advanced PII detection with NER models
- Set up real-time monitoring dashboards
- Create approval workflow system

---

**Remember**: Guardrails are not optional - they're essential for production AI systems!

---
# Exercises 🎯

## Exercise 1: Enhanced PII Detection
Extend the PII detection to include:
- Bank account numbers (Indian format)
- PAN card numbers
- Passport numbers
- Credit card numbers

Test with sample data.

## Exercise 2: Role-Based Access Control
Implement a guardrail that:
- Employees can only view their own data
- Managers can view their team's data
- HR admins can view all data

## Exercise 3: Content Moderation
Create a guardrail that uses an LLM to:
- Detect inappropriate requests
- Classify request sensitivity
- Provide alternative responses

## Exercise 4: Audit Trail System
Build a comprehensive audit system that:
- Logs all tool executions
- Tracks who accessed what data
- Generates compliance reports
- Alerts on suspicious patterns

## Bonus Challenge: Multi-Layer Guardrail System
Create a production-ready system with:
- 5+ different guardrail types
- Configurable guardrail policies
- Real-time monitoring dashboard
- Automated incident response
- Integration with external approval systems